##### <span style="color:red">Name:</span>

=== *Edit this code cell to include your name and DuckID* ===

# <span style="color:teal;">CIS 211 Project 8:  &nbsp; Weather Service</span>

##### Due 11:00 P.M. Friday June 5

##### Reading:  Perkovic Sec 11.1 -- 11.2

The project this week uses "REST" style web services to create a weather report. 

### `weather` 

Below you will find the defintion of a function named `weather` that prints a weather forecast for a specified location.

There are three different ways to call the function.  One is to pass a zip code:
<pre>
   weather(zip = 97405)
</pre>

A second way to print a forecast is to pass an address to the function:
<pre>
   weather(address = '1477 E 13th Ave, Eugene, OR')
</pre>

The third option is to pass latitude and longitude coordinates:
<pre>
   weather(lat = 44.0, long = -123.0)
</pre>

In each case the result will be the same, the current weather forecast for the specified location:
<pre>
   Showers likely and possibly a thunderstorm.  Cloudy, with a high near 63. Calm wind becoming 
   west southwest 5 to 7 mph in the afternoon.  Chance of precipitation is 70%. New rainfall 
   amounts between a tenth and quarter of an inch, except higher amounts possible in thunderstorms. 
</pre>

**Note:** The lecture notes for this week will explain how to write a function definition that allows users to call the function using keywords and values.

You have three jobs this week:
1. Write a helper function called `get_forecast` that will use a web service provided by NOAA to fetch the current weather forecast for a specified location.  
1. Since the NOAA service only works if you pass it a latitude and longitude, you need to write a second helper function, called `get_coords`, will look up a latitude and longitude given a zip code or address.
1. You need to test the top level `weather` function and explain how it handles errors.

### NOAA API 

A template for the URL for a web service provided by NOAA is
<pre>
http://forecast.weather.gov/MapClick.php?lat={}&lon={}&FcstType=json
</pre>
When numbers specifying latitude and longitude are substituted into this template the URL will return a JSON string with a complete forecst for that location.

For example, this shell command will fetch a weather report for Eugene:
<pre>
   % curl 'http://forecast.weather.gov/MapClick.php?lat=44.0&lon=-123.0&FcstType=json'
</pre>

When the result is translated as a JSON object you will get a Python dictionary.  One of the items in the dictionary is called `data`, which is itself a dictionary.  Inside that dictionary is an item called `text`, which is a list of strings.  The weather prediction you want is the first string in that list.

For example, if you save the result of the call to `json.loads` in a dictionary named `forecast`, this expression will get the string containing the prediction for the next few hours:
<pre>
forecast['data']['text'][0]
</pre>

### Geocoder API 

The NOAA service expects a location to be specified by its geographic coordinates (latitude and longitude).  If `weather` is called with a zip code or address we can use a service called a "geocoder" to look up the latitude and longitude.

There are two APIs for this service.  Use the first one if you have a zip code, use the second if you have an address:
<pre>
'http://geocoder.us/service/json?zip={}'
'http://geocoder.us/service/json?address={}&parse_address=1'
</pre>

For example, both of these shell commands return a JSON string containing the coordinates of the CS department:
<pre>
   % curl 'http://geocoder.us/service/json?zip=97403'
   % curl 'http://geocoder.us/service/json?address=1477+E.+13th+Ave.+Eugene,+OR&parse+address=1'
</pre>

When this JSON string is converted into a Python object you will get a list (some calls may specifiy ambiguous addresses or multiple zip codes).  You can assume the first item is the one you want.  This item will be a dictionary, and the coordinates are in the `lat` and `long` components of this dictionary.  

For example, if the result of the call to `json.loads` is saved in a list called `coords`, these expressions will look up the latitude and longitude of the specified location:
<pre>
coords[0]['lat']
coords[0]['long']
</pre>

### Top Level Function 

The following code cell contains the `import` statements for the Python libraries needed by this project and the definition of the top level `weather` function.

In [146]:
from urllib.request import urlopen
from urllib.error import HTTPError
import json


def weather(**spec):
    try:
        if 'zip' in spec or 'address' in spec:
            lat, long = get_location(**spec)
        elif not ('lat' in spec and 'long' in spec):
            raise Exception('specify zip, address, or latitude and longitude')
        else:
            lat, long = spec['lat'], spec['long']
        print(get_forecast(lat, long))
    except HTTPError:
        print('Incomplete or incorrect specification', spec)
    except Exception as err:
        print(err)

### Project 1:   `get_forecast` 

Write the function definition for the `get_forecast` helper function.  The funtion should take two integer arguments, the latitude and longitude, and return a single string, the weather forecast for that location.

##### <span style="color:red">Description:</span>

Uses input to concat string for open url to retrive json.  Decodes and returns forecast

##### <span style="color:red">Program:</span>

In [1]:
# == use this code cell to define get_forecast ==
def get_forecast(lat, long):
    '''
    Arguments: lat long
    returns: json forecast value
    
    '''
    url= 'http://forecast.weather.gov/MapClick.php?lat='+str(lat)+'&lon='+str(long)+'&FcstType=json'
    
    response = urlopen(url)
    data = response.read()
    r=json.loads(data.decode('utf8'))
    return r['data']['text'][0]

##### <span style="color:red">Tests:</span>

In [95]:
# == use one or more code cells to test get_forecast ==
get_forecast(44.0,-123.0)

A 50 percent chance of showers, mainly before midnight.  Cloudy, with a low around 50. West wind 3 to 7 mph. 


### Project 2:  `get_location` 

Write the function definition for the `get_location` helper function.  This function should allow users to either pass a zip code or an address, and the return value should be a pair of numbers corresponding to the latitude and longitude of the specified location.

**Note** This week's lecture notes will explain how to write a function that can be passed keyword arguments.  The top level `weather` function is an example -- this function can be passed arguments named `zip`, `address`, or `lat` and `lon`.


##### <span style="color:red">Description:</span>

Uses keyword argument.  Checks if zip or address is used.  Uses appropriate url for geocoder. Returns lat long

##### <span style="color:red">Program:</span>

In [2]:
# == use this code cell to define get_location ==
def get_location(**spec):
    '''
    Arguments: keyword spec
    returns: lat long
    
    '''
    if 'zip' in spec:
        url= 'http://geocoder.us/service/json?zip=' + str(spec['zip'])
        response = urlopen(url)
        data = response.read()
        r=json.loads(data.decode('utf8'))
        return r[0]['lat'],r[0]['long']
    if 'address' in spec:
        address=str(spec['address'])
        address=address.replace(' ','+')
  
        url= 'http://geocoder.us/service/json?address=' + address + '&parse+address=1'
        response = urlopen(url)
        data = response.read()
        r=json.loads(data.decode('utf8'))
        return r[0]['lat'],r[0]['long']


##### <span style="color:red">Tests:</span>

In [152]:
# == use one or more code cells to test get_location ==
get_location(zip=97401)

('44.06092', '-123.08275')

In [153]:
weather(address='1840 mill street eugene or 97401')

A 40 percent chance of showers, mainly before 11pm.  Cloudy, with a low around 52. West wind 3 to 7 mph. 
None


### Project 3:  Test the `weather` Function 

In the space below use one or more code cells to make calls to `weather`.  Note that if you pass `lat` and `long` as arguments you can call `weather` even if you have not yet implemented the `get_location` helper.

Your tests should call `weather` with zip codes, addresses, and latitude/longitude.

After the code cells, write a brief description of how the function handles errors.  What happens if you pass an illegal or unknown zip code?  If you don't pass any valid arguments?